In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [7]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [9]:
from bailian_nlp.modules.data.tokenization import BailianTokenizer

tokenizer = BailianTokenizer(vocab_file)

text = '这是个好人。are you ok Málaga'

tokenizer.tokenize(text)

(['这', '是', '个', '好', '人', '。', 'are', 'you', 'ok', '[UNK]'],
 [(1, (0, 1)),
  (1, (1, 2)),
  (1, (2, 3)),
  (1, (3, 4)),
  (1, (4, 5)),
  (1, (5, 6)),
  (1, (6, 9)),
  (0, (9, 10)),
  (1, (10, 13)),
  (0, (13, 14)),
  (1, (14, 16)),
  (0, (16, 17)),
  (1, (17, 22)),
  (0, (22, 23))])

In [67]:
unicodedata.normalize('NFD', text)

'a á'

In [ ]:
from bailian_nlp.modules.data.tokenization import BailianTokenizer
tokenizer = BailianTokenizer(vocab_file)
text = 'áf 3 are you '
text = 'http://www.xteacher.com'
text = '中华人'
text = '今年在韩国举办的「二十一世纪的计算」学术研讨会以人机协作（HumanandMachineWorkingasaTeam）为主题，现任微软全球资深副总裁，微软亚太研发集团主席，兼微软亚洲研究院院长洪小文博士做了题目为《人工智能与人类智能的共同进化》（Co-EvolutionofArtificialIntelligenceandHumanIntelligence）的演讲。'
tokens, marker = tokenizer.tokenize(text)
# 切词
print(len(text), tokens, marker)
# 还原
for i,(_, (st, ed)) in enumerate(marker):
    print(i, text[st:ed])


In [116]:
tokenizer.recover_text(text, tokens, ['x', 's', '3'], marker)

[('中', 'x'), ('华', 's'), ('人', '3')]

In [10]:
raw_data_dir = os.path.dirname(data_path)
seg_file = os.path.join(raw_data_dir, 'final_baidu-23w.txt')
seg_f = open(seg_file)

pos_sents = seg_f.readlines()
pos_sents[0]

'近日/t ，/w 上海未来伙伴机器人有限公司/nt CEO/ti 费旭锋/nr 表示/v 随着/p 人工智能/nz 时代/n 的/u 来临/vn ，/w 教育机器人/nz 蕴含/v 的/u 巨大/a 价值/n 也/d 日益/d 凸显/v ，/w 作为/p 训练/v 少年儿童/n 成功/a 能力/n 、/w 提升/v 科技/n 素养/n 的/u 最佳/a 平台/n ，/w 其/r 市场/n 规模/n 正/d 逐渐/d 扩大/v ，/w 预计/v 全球/n 教育/vn 机器人/n 市场/n 容量/n 在/p 2025年/t 将/d 超过/v 3000亿元/m 。/w\n'

In [16]:
from pytorch_pretrained_bert import tokenization
tokenization._is_punctuation('\n')

False

In [12]:
idx = 0
counter = 0
while idx < len(pos_sents):
    try:
        idx += 1
        pos_sent = pos_sents[idx]
        tokenizer = BailianTokenizer(vocab_file)
        tokenizer.tokenize_with_pos_text(pos_sent)
    except:
        counter += 1
        print(counter, idx, counter/(idx+1))
#         raise

counter

1 211 0.0047169811320754715
2 1471 0.001358695652173913
3 2123 0.0014124293785310734
4 2985 0.0013395847287340924
5 3178 0.0015728216420257944
6 3179 0.0018867924528301887
7 3520 0.0019880715705765406
8 4001 0.001999000499750125
9 4114 0.002187120291616039
10 4115 0.0024295432458697765
11 4703 0.00233843537414966
12 4704 0.0025504782146652497
13 5262 0.002470074102223067
14 5263 0.0026595744680851063
15 5268 0.0028468400075915734
16 5269 0.0030360531309297912
17 5386 0.00315574531279005
18 5440 0.003308215401580592
19 6305 0.0030130034887408817
20 7308 0.0027363524421945545
21 8462 0.0024813895781637717
22 8463 0.0025992438563327033
23 8464 0.0027170702894270525
24 8852 0.0027109454422229754
25 9766 0.0025596396027439336
26 9966 0.0026086084077455605
27 9967 0.002708667736757624
28 10678 0.0026219683490963574
29 11635 0.002492265383293228
30 11902 0.0025203730152062506
31 12450 0.002489759858645892
32 12618 0.002535858625881607
33 14194 0.0023247622402254313
34 14607 0.0023274917853231

686

In [ ]:
idx, counter

In [152]:
pos_sent

'今年/t 在/p 韩国/ns 举办/v 的/u 「/w 二十一世纪/t 的/u 计算/vn 」/w 学术/n 研讨会/n 以/p 人机/n 协作/v （/w Human/xc and/xc Machine/xc Working/xc as/xc a/xc Team/n ）/w 为主题/v ，/w 现任/v 微软/nt 全球/n 资深副总裁/ti ，/w 微软亚太研发集团/nt 主席/ti ，/w 兼/v 微软亚洲研究院/nt 院长/ti 洪小文/nr 博士/ti 做/v 了/u 题目/n 为/v 《/w 人工智能/nz 与/c 人类/n 智能/n 的/u 共同/a 进化/vn 》/w （/w Co/nz -/w Evolution/nt of/xc Artificial/xc Intelligence/xc and/xc Human/xc Intelligence/xc ）/w 的/u 演讲/vn 。/w\n'

In [168]:
tokenizer.tokenize_with_pos_text(pos_sents[idx-1])

(['香',
  '港',
  '科',
  '技',
  '大',
  '学',
  '教',
  '授',
  '、',
  '第',
  '四',
  '范',
  '式',
  '首',
  '席',
  '科',
  '学',
  '家',
  '杨',
  '强',
  '在',
  '雷',
  '锋',
  '网',
  '承',
  '办',
  '的',
  'cc',
  '##f',
  '-',
  'ga',
  '##ir',
  '全',
  '球',
  '人',
  '工',
  '智',
  '能',
  '与',
  '机',
  '器',
  '人',
  '峰',
  '会',
  '大',
  '会',
  '上',
  '，',
  '为',
  '大',
  '家',
  '讲',
  '述',
  '了',
  '人',
  '工',
  '智',
  '能',
  '要',
  '取',
  '得',
  '成',
  '功',
  '应',
  '当',
  '具',
  '备',
  '的',
  '五',
  '个',
  '必',
  '要',
  '条',
  '件',
  '和',
  '迁',
  '移',
  '学',
  '习',
  '的',
  '本',
  '质',
  '。'],
 ['B_t',
  'I_t',
  'I_t',
  'I_t',
  'I_t',
  'E_t',
  'B_w',
  'E_w',
  'S_n',
  'B_t',
  'I_t',
  'I_t',
  'E_t',
  'B_n',
  'I_n',
  'I_n',
  'I_n',
  'E_n',
  'B_p',
  'E_p',
  'S_n',
  'B_v',
  'I_v',
  'E_v',
  'B_u',
  'E_u',
  'S_n',
  'S_n',
  'S_w',
  'S_v',
  'B_n',
  'E_n',
  'B_n',
  'E_n',
  'B_c',
  'I_c',
  'I_c',
  'E_c',
  'S_n',
  'B_n',
  'I_n',
  'E_n',
  'B_n',
  'E_n',
  'B_f',
  'E_f

In [4]:
def build_data():
    
    import re
    
    p = re.compile(r'(.+?)/(?:([a-z]{1,2})(?:$| ))')
    
    raw_data_dir = os.path.dirname(data_path)
    seg_file = os.path.join(raw_data_dir, 'final_baidu-23w.txt')
    fake_file = os.path.join(raw_data_dir, 'fake.txt')
    special_file = os.path.join(raw_data_dir, 'special.txt')
    dict_file = os.path.join(raw_data_dir, 'single.txt')
    
    delimiter='△△△'
    
    replace_chars = [
        '\x97',
        '\uf076',
        "\ue405",
        "\ue105",
        "\ue415",
        '\x07',
        '\x7f',
        '\u3000',
        '\xa0',
        ' '
    ]
    with open(seg_file) as fin1, \
          open(fake_file) as fin2, \
          open(special_file) as fin3, \
          open(dict_file) as fin4, \
          open(train_path, 'w') as train_f, \
          open(valid_path, 'w') as valid_f:
        
        train_f.write(f'0{delimiter}1\n')
        valid_f.write(f'0{delimiter}1\n')
        
        fins = [fin1, fin2, fin3, fin4]
        for k, fin in enumerate(fins):
            for line in fin:
                line = line.strip()
                if not line:
                    continue

                import random
                score = random.random()

                if k < 2:
                    fout = train_f if score > 0.006 else valid_f
                else:
                    fout = train_f
                
                words = []
                flags = []
                for word, flag in p.findall(line):
                    from bailian_nlp.modules.data.tokenization import _is_control
                    
                    char_list = [
                        c for c in list(word)
                        if not (c in replace_chars or c.isspace() or _is_control(c))
                    ]
                    char_size = len(char_list)
                    if char_size == 1:
                        # 一些错误的单个字符实体剔除掉
                        if flag in ['nt', 'ti', 'nr', 'ns', 'nz']:
                            flag = 'xx'
                        # 单个
                        tag_list = [f'S_{flag}']
                    elif char_size == 0:
                        continue
                    else:
                        tag_list = [f'B_{flag}'] + [f'I_{flag}']  * (len(char_list) - 2) + [f'E_{flag}']

                    if char_size != len(tag_list) or char_size == 0:
                        print(line)
                        print(word, flag)
                        print(char_list, tag_list)
                        

                    words.extend(char_list)
                    flags.extend(tag_list)

                assert len(words) == len(flags)

                fout.write(delimiter.join([
                    ' '.join(flags),
                    ' '.join(words)
                ]))
                fout.write('\n')
            
            
            
build_data()
    

In [ ]:
# 正常训练

from bailian_nlp.modules import BertNerData as NerData

data = NerData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    is_cls=False,
    max_seq_len=128,
    batch_size=128
    
)

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    bert_config_file, 
    init_checkpoint_pt,
    enc_hidden_dim=256
)
model.get_n_trainable_params()


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


2019-04-18 11:29:37,496 DEBUG: get_data cost 324.762294s


2019-04-18 11:29:39,012 DEBUG: get_data cost 1.514747s


2019-04-18 11:29:39,856 INFO: Resuming train... Current epoch 0.


2019-04-18 12:35:36,446 INFO: 
epoch 1, average train epoch loss=39.17



2019-04-18 12:35:50,216 INFO: on epoch 0 by max_f1: 0.905
2019-04-18 12:35:50,217 INFO: Saving new best model...


...

2019-04-18 17:57:00,977 INFO: on epoch 5 by max_f1: 0.935
2019-04-18 17:57:00,978 INFO: Saving new best model...


              precision    recall  f1-score   support

       <pad>      0.000     0.000     0.000         0
       [CLS]      1.000     1.000     1.000      2617
         B_t      0.978     0.971     0.975      1252
         E_t      0.977     0.970     0.974      1238
         S_w      0.999     0.998     0.998      9860
        B_nt      0.939     0.965     0.952      4923
        I_nt      0.951     0.977     0.964     21804
        E_nt      0.934     0.935     0.935      4845
        B_ti      0.982     0.990     0.986      3104
        I_ti      0.942     0.997     0.969      4430
        E_ti      0.986     0.954     0.970      3078
        B_nr      0.958     0.971     0.964      3842
        I_nr      0.871     0.953     0.911      2967
        E_nr      0.952     0.909     0.930      3818
         B_v      0.943     0.931     0.937      5713
         E_v      0.947     0.931     0.939      5679
         B_p      0.964     0.958     0.961       311
         E_p      0.964    

2019-04-18 18:57:05,052 INFO: 
epoch 7, average train epoch loss=2.1653



2019-04-18 18:57:18,359 INFO: on epoch 6 by max_f1: 0.936
2019-04-18 18:57:18,360 INFO: Saving new best model...


              precision    recall  f1-score   support

       <pad>      0.000     0.000     0.000         0
       [CLS]      1.000     1.000     1.000      2617
         B_t      0.978     0.975     0.976      1252
         E_t      0.977     0.975     0.976      1238
         S_w      0.999     0.998     0.998      9860
        B_nt      0.938     0.967     0.953      4923
        I_nt      0.954     0.977     0.965     21804
        E_nt      0.934     0.939     0.936      4845
        B_ti      0.981     0.991     0.986      3104
        I_ti      0.940     0.998     0.968      4430
        E_ti      0.985     0.956     0.970      3078
        B_nr      0.959     0.973     0.966      3842
        I_nr      0.878     0.959     0.917      2967
        E_nr      0.953     0.911     0.932      3818
         B_v      0.948     0.928     0.938      5713
         E_v      0.951     0.928     0.939      5679
         B_p      0.971     0.955     0.963       311
         E_p      0.971    

In [ ]:
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')

In [5]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 1
learner.load_model()
learner.t_total = num_epochs * len(data.train_dl)
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


2019-04-17 10:47:39,699 INFO: load default user_dict in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/dict/user_dict.txt
2019-04-17 10:47:39,702 INFO: 本次加载词条数：3
2019-04-17 10:47:39,703 INFO: 当前总词条数: 3
2019-04-17 10:47:47,988 INFO: found pos model file in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
2019-04-17 10:47:48,867 INFO: pos model loads success!


2019-04-17 10:51:25,215 DEBUG: get_data cost 211.709043s


2019-04-17 10:51:26,159 DEBUG: get_data cost 0.94289s


2019-04-17 10:51:26,982 INFO: found pos model file in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
2019-04-17 10:51:27,241 INFO: pos model loads success!
2019-04-17 10:51:27,492 INFO: Resuming train... Current epoch 0.


2019-04-17 12:10:04,434 INFO: 
epoch 1, average train epoch loss=1.4911



2019-04-17 12:10:14,770 INFO: on epoch 0 by max_f1: 0.928
2019-04-17 12:10:14,772 INFO: Saving new best model...


              precision    recall  f1-score   support

        E_an      0.821     0.780     0.800        41
         B_t      0.987     0.981     0.984       851
       [CLS]      1.000     1.000     1.000      2574
        E_ti      0.981     0.930     0.955      1763
        I_xx      0.000     0.000     0.000        34
         B_q      0.667     0.500     0.571         4
        E_nr      0.960     0.864     0.909      2683
         B_v      0.918     0.937     0.928      3055
           X      1.000     1.000     1.000       741
        B_xx      0.000     0.000     0.000        12
        S_xc      0.815     0.898     0.855        59
         I_j      0.000     0.000     0.000         0
         S_u      0.996     0.995     0.995      1533
        E_ad      0.867     0.903     0.884       144
        S_nw      0.000     0.000     0.000         0
         I_s      0.833     0.833     0.833         6
        I_nr      0.838     0.965     0.897      2184
         I_t      0.986    

In [6]:
learner.fit(num_epochs, target_metric='f1')

2019-04-17 12:52:44,550 INFO: Resuming train... Current epoch 1.


2019-04-17 14:11:02,057 INFO: 
epoch 2, average train epoch loss=1.7876



2019-04-17 14:11:12,114 INFO: on epoch 1 by max_f1: 0.932
2019-04-17 14:11:12,115 INFO: Saving new best model...


              precision    recall  f1-score   support

        E_an      0.806     0.707     0.753        41
         B_t      0.981     0.987     0.984       851
       [CLS]      1.000     1.000     1.000      2574
        E_ti      0.981     0.934     0.957      1763
        I_xx      0.842     0.471     0.604        34
         B_q      1.000     0.500     0.667         4
        E_nr      0.960     0.858     0.906      2683
         B_v      0.945     0.921     0.933      3055
           X      1.000     1.000     1.000       741
        B_xx      0.200     0.083     0.118        12
        S_xc      0.806     0.915     0.857        59
         I_j      0.000     0.000     0.000         0
         S_u      0.996     0.995     0.995      1533
        E_ad      0.902     0.896     0.899       144
        S_nw      0.000     0.000     0.000         0
         I_s      1.000     0.833     0.909         6
        I_nr      0.834     0.960     0.893      2184
         I_t      0.987    

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [3]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


2019-04-18 11:13:32,906 INFO: load default user_dict in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/dict/user_dict.txt
2019-04-18 11:13:32,908 INFO: 本次加载词条数：3
2019-04-18 11:13:32,909 INFO: 当前总词条数: 3
2019-04-18 11:13:40,573 INFO: found pos model file in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
2019-04-18 11:13:40,888 INFO: pos model loads success!


In [7]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
# text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
text = '董事'
# text = '一言九鼎'
# text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res

2019-04-18 11:17:00,052 DEBUG: get_data cost 0.050898s


2019-04-18 11:17:00,178 DEBUG: text_array_for_predict cost 0.18011s



0.18120479583740234


[[('2012-11-8 ', 't'),
  ('的', 'u'),
  ('客户', 'n'),
  ('包括', 'v'),
  ('：', 'w'),
  ('宝洁 ', 'nt'),
  ('、', 'w'),
  ('杜邦', 'nt'),
  ('、', 'w'),
  ('强生', 'nt'),
  ('、', 'w'),
  ('百事可 乐', 'nt'),
  ('、', 'w'),
  ('3M ', 'nz'),
  ('、', 'w'),
  ('三菱', 'nt'),
  ('、', 'w'),
  ('吉列', 'nz'),
  ('、', 'w'),
  ('通用医疗', 'nt'),
  ('系统', 'n'),
  ('、', 'w'),
  ('美国 运通', 'nt'),
  ('、', 'w'),
  ('雅芳', 'nt'),
  ('、', 'w'),
  ('纽约银 行', 'nt'),
  ('、', 'w'),
  ('百时美', 'nt'),
  ('—', 'w'),
  ('施贵 宝', 'nr'),
  ('、', 'w'),
  ('礼来', 'nz'),
  ('、', 'w'),
  ('迪斯尼', 'nt'),
  ('、', 'w'),
  ('纳贝斯克', 'ns'),
  ('、', 'w'),
  ('纽约', 'ns')]]